In [1]:
from dotenv import load_dotenv, find_dotenv
import chromadb
load_dotenv(find_dotenv())

True

In [16]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import MathpixPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

In [40]:
# method 1 of loading pdf
loader = PyPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf")
pages = loader.load_and_split()

In [4]:
# method 2 of loading pdf
loader = UnstructuredPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf")
# loader = MathpixPDFLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.pdf")
data = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\angad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [20]:
loader = UnstructuredMarkdownLoader(r"C:\Users\angad\OneDrive\Desktop\AML\thesis\MSc_Thesis_02298815.md")
data = loader.load()

In [23]:
data[0]

Document(page_content='Federated Learning for Short-Term\n\nLoad Forecasting\n\nAuthor\n\nA. Khurana\n\nCID: 02298815 Supervised by\n\nDr Fei Teng\n\nA Thesis submitted in fulfillment of requirements for the degree of\n\nMaster of Science in Applied Machine Learning\n\nDepartment of Electrical and Electronic Engineering\n\nImperial College London\n\n2023\n\nAbstract\n\nElectricity is the backbone of today’s society. It powers everything from houses to industries and businesses. The core infrastructure behind electricity is the grid which requires a continual balance between supply and demand for optimal functioning. This necessitates the accurate short-term forecasting of the load (STLF) consumed. In recent years, deep learning has been employed with federated learning for STLF as federated learning allows collaboration of clients without the need to aggregate their data which is difficult due to privacy-related and logistical concerns.\n\nThis study analyses the impact of the network 

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    add_start_index = True,
)

In [5]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name='angad_embedding')

In [6]:
collection

Collection(name=angad_embedding)

In [7]:

# texts = text_splitter.split_documents(pages)
# documents = [i.page_content for i in texts]
# metadatas = [i.metadata for i in texts]
# ids = [str(i) for i in range(len(documents))]
# collection.add(
# documents = documents,
# metadatas = metadatas,
# ids = ids
#     )

In [30]:
collection.get(0)['documents'][18]

'12 CHAPTER 1. INTRODUCTION\nas ambient weather and time of day. To incorporate such variables, multivariate linear regression\nknown as multiple regression was used [2]. However, load data has inherent temporal which are\nnot exploited using regression techniques. ARIMA (AutoRegressive Integrated Moving Average)\nmethods have shown increased performance accuracies for load forecasting compared to regression\n[3]. The multivariant version of ARIMA known as ARIMAX incorporates the external factors into\nthe forecasting [4]. A thorough investigation into the statistical methods is conducted in Chapter\n2. These techniques assume linearity, but deep learning offers better forecasting accuracy through\nnon-linear modeling.\nDeep learning is a branch of machine learning that uses neural networks with multiple layers\nto automatically learn and extract complex patterns and representations from data, enabling the\ndevelopment of advanced AI systems capable of tasks like image recognition and 

In [37]:
pages[0]

Document(page_content='Federated Learning for Short-Term\nLoad Forecasting\nAuthor\nA. Khurana\nCID: 02298815\nSupervised by\nDr Fei Teng\nA Thesis submitted in fulfillment of requirements for the degree of\nMaster of Science in Applied Machine Learning\nDepartment of Electrical and Electronic Engineering\nImperial College London\n2023', metadata={'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf', 'page': 0})

In [8]:

documents = [i.page_content for i in pages]
metadatas = [i.metadata for i in pages]
ids = [str(i) for i in range(len(documents))]
collection.add(
documents = documents,
metadatas = metadatas,
ids = ids
    )

In [35]:
# question = "Who is my supervisor?"
question = ""
results = collection.query(query_texts = [question], n_results = 5)

In [36]:
results

{'ids': [['20', '34', '3', '7', '5']],
 'distances': [[1.2122215032577515,
   1.3455324172973633,
   1.387810230255127,
   1.5093117952346802,
   1.6102261543273926]],
 'metadatas': [[{'page': 21,
    'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf'},
   {'page': 35,
    'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf'},
   {'page': 4,
    'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf'},
   {'page': 8,
    'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf'},
   {'page': 6,
    'source': 'C:\\Users\\angad\\OneDrive\\Desktop\\AML\\thesis\\MSc_Thesis_02298815.pdf'}]],
 'embeddings': None,
 'documents': [['14 CHAPTER 1. INTRODUCTION',
   '218 CHAPTER 2. LITERATURE REVIEW',
   'Declaration of Originality\nI hereby declare that the work presented in this thesis is my own unless otherwise stated. To the\nbest of my knowledge the work is original a

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

results = collection.query(query_texts = [question], n_results = 2)

template = """The following piece of text is given: {text}. Please answer any following questions ONLY using THIS piece of text in a brief manner."""
human_template = "{question}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])
chain = chat_prompt | ChatOpenAI() 
context = ''
for i in range(len(results['documents'])):
    context = context + results['documents'][0][i] + '\n'
chain.invoke({"text": context, "question": question})


AIMessage(content='I apologize, but as an AI language model, I am unable to physically fetch or retrieve specific documents for you. However, if you have the text of your introduction chapter, I can help answer any questions you may have about it.')

In [12]:
# from langchain.llms import OpenAI
# from langchain.chat_models import ChatOpenAI

# llm = OpenAI(temperature=0.1);
# chat_model = ChatOpenAI();
# llm.predict("hi! how are you?")

In [13]:
# from langchain.prompts.chat import ChatPromptTemplate

# template = "You are a helpful assistant that translates {input_language} to {output_language}."
# human_template = "{text}"

# chat_prompt = ChatPromptTemplate.from_messages([
#     ("system", template),
#     ("human", human_template),
# ])

# x = chat_prompt.format_messages(input_language="English", output_language="French", text="Okay")